# Большая домашка 1: классификация отзывов

## Данные

Скачиваются по [ссылке](https://drive.google.com/open?id=1gJEpwM3zdV5xe9nRU0VwQmGTiOTQdp10). Это отзывы пользователей на фильмы с КиноПоиска.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('reviews.csv')

In [5]:
df.head()

,label,text
0,1,Эпиграф Добро которое ты делаешь от сердца ты ...
1,1,Теперь это один из моих любимых фильмов в жанр...
2,1,Что скрыто в фильме Лучше не бывает Одна шикар...
3,1,Перед нами очень милое и доброе кино которое л...
4,1,Завязка Мелвин Удал популярный писатель Нет не...


In [6]:
set(df.label)

{-1, 0, 1}

В колонке `label` три класса: положительный (1), нейтральный (0) и отрицательный (-1).

## Baseline

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
vec = CountVectorizer()
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.label)

In [21]:
nb = MultinomialNB()
clf = nb.fit(X_train, y_train)

In [20]:
from sklearn.metrics import classification_report

In [22]:
print(classification_report(y_test, clf.predict(X_test)))

             precision    recall  f1-score   support

         -1       0.67      0.20      0.31      1094
          0       0.33      0.07      0.12      1347
          1       0.82      0.98      0.89      9185

avg / total       0.75      0.80      0.75     11626



## Задание

Побейте baseline выше как минимум тремя способами.


**Обязательно**:
* попробовать разный препроцессинг (2 балла)
* попробовать разные классификаторы (2 балла)
* поподбирать гиперпараметры классификаторов, можно с использованием GreedSearch (2 балла)
* попробуйте объяснить полученные результаты (2 балла)
* визуализируйте веса признаков для лучшей модели (если лучшая модель не выдаёт коэффициенты признаков, возьмите логистическую регрессию); какие признаки оказались самыми значимыми? (2 балла)

Если вы перебрали уже много разных способов и выполнили все обязательные критерии, но побить baseline тремя способами так и не получилось, не беспокойтесь. Главное -- полнота попыток :)

Good luck and have fun! ;)

### Бонус
(+2 балла к основному заданию; оценка за всё задание может быть выше 10)

Придумайте ещё 2-3 осмысленных споцоба визуализации данных или результатов. Добавьте их ниже или в текст основного задания. и сошлитесь на них здесь.